## Splink data linking demo (link only)

In this demo we link two small datasets.  

We assume we have a list of people in one table who we want to find in a larger table.  It is assumed that due to transcription or other errors, there will often not be an exact match.

The larger table contains duplicates, but in this notebook we use the `link_only` setting, so `splink` makes no attempt to deduplicate these records.    Note it is possible to simultaneously link and dedupe using the `link_and_dedupe` setting.

**Important** Where deduplication is not required, `link_only` can provide an important performance boost by dramatically reducing the number of records which need to be compared.

For example, if you wanted to link 10 records to 1,000, then the maximum number of comparisons that need to be made (i.e. with no blocking rules) is 10,000.  If you need to dedupe as well, that number would be n(n-1)/2 = 509,545.

I print the output at each stage using `spark_dataframe.show()`.  This is for instructional purposes only - it degrades performance and shouldn't be used in a production setting.

## Step 1:  Imports and setup

The following is just boilerplate code that sets up the Spark session and sets some other non-essential configuration options

In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f

In [2]:
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime

#Extracting the correct URL from hive-site.xml
tree = ET.parse('/etc/hadoop/conf/hive-site.xml')
root = tree.getroot()

for prop in root.findall('property'):
    if prop.find('name').text == "hive.metastore.warehouse.dir":
        storage = prop.find('value').text.split("/")[0] + "//" + prop.find('value').text.split("/")[2]

print("The correct Cloud Storage URL is: {}".format(storage))

os.environ['STORAGE'] = storage

The correct Cloud Storage URL is: s3a://demo-aws-2


In [3]:
#conf=SparkConf()

# Load in a jar that provides extended string comparison functions such as Jaro Winkler.
# Splink
#     conf.set('spark.driver.extraClassPath', 'jars/scala-udf-similarity-0.0.6.jar,jars/graphframes-0.6.0-spark2.3-s_2.11.jar')
#     conf.set('spark.jars', 'jars/scala-udf-similarity-0.0.6.jar,jars/graphframes-0.6.0-spark2.3-s_2.11.jar')
#conf.set('spark.driver.extraClassPath', 'jars/scala-udf-similarity-0.0.6.jar')
#conf.set('spark.jars', 'jars/scala-udf-similarity-0.0.6.jar')
#conf.set('spark.jars.packages', 'graphframes:graphframes:0.6.0-spark2.3-s_2.11')

#sc = SparkContext.getOrCreate(conf=conf)
#sc.setCheckpointDir("temp_graphframes/")


spark = SparkSession\
    .builder\
    .appName("Entity Resolution with Lineage")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-1")\
    .config("spark.yarn.access.hadoopFileSystems", os.environ['STORAGE'])\
    .config("spark.driver.extraClassPath", "jars/scala-udf-similarity-0.0.6.jar")\
    .config("spark.jars", "jars/scala-udf-similarity-0.0.6.jar")\
    .getOrCreate()

# Register UDFs
from pyspark.sql import types
spark.udf.registerJavaFunction('jaro_winkler_sim', 'uk.gov.moj.dash.linkage.JaroWinklerSimilarity', types.DoubleType())
spark.udf.registerJavaFunction('Dmetaphone', 'uk.gov.moj.dash.linkage.DoubleMetaphone', types.StringType())

In [4]:
spark

In [5]:
import pandas as pd 
pd.options.display.max_columns = 500

In [6]:
import logging 
logging.basicConfig()  # Means logs will print in Jupyter Lab

# Set to DEBUG if you want splink to log the SQL statements it's executing under the hood
logging.getLogger("splink").setLevel(logging.INFO)

## Step 2: Read in the data

The `l` and `r` stand for 'left' and 'right.  It doesn't matter which of the two datasets you choose as the left, performance and results will be the same.

⚠️ Note that `splink` makes the following assumptions about your data:

-  There is a field containing a unique record identifier in each dataset
-  The two datasets being linked have common column names - e.g. date of birth is represented in both datasets in a field of the same name.   In many cases, this means that the user needs to rename columns prior to using `splink`


In [7]:
df_l = spark.read.parquet("data/fake_df_l.parquet")
df_r = spark.read.parquet("data/fake_df_r.parquet")
df_l.show(5)
df_r.show(5)

+---------+----------+-------+----------+------------+--------------------+-----+
|unique_id|first_name|surname|       dob|        city|               email|group|
+---------+----------+-------+----------+------------+--------------------+-----+
|        0|    Julia |   null|2015-10-29|      London| hannah88@powers.com|    0|
|        4|      oNah| Watson|2008-03-23|      Bolton|matthew78@ballard...|    1|
|       13|    Molly |   Bell|2002-01-05|Peterborough|                null|    2|
|       15| Alexander|Amelia |1983-05-19|     Glasgow|ic-mpbell@alleale...|    3|
|       20|    Ol vri|ynnollC|1972-03-08|    Plymouth|derekwilliams@nor...|    4|
+---------+----------+-------+----------+------------+--------------------+-----+
only showing top 5 rows

+---------+----------+-------+----------+------+--------------------+-----+
|unique_id|first_name|surname|       dob|  city|               email|group|
+---------+----------+-------+----------+------+--------------------+-----+
|        

## Step 3:  Configure splink using the `settings` object

Most of `splink` configuration options are stored in a settings dictionary.  This dictionary allows significant customisation, and can therefore get quite complex.  

💥 We provide an tool for helping to author valid settings dictionaries, which includes tooltips and autocomplete, which you can find [here](http://robinlinacre.com/splink_settings_editor/).

Customisation overrides default values built into splink.  For the purposes of this demo, we will specify a simple settings dictionary, which means we will be relying on these sensible defaults.

To help with authoring and validation of the settings dictionary, we have written a [json schema](https://json-schema.org/), which can be found [here](https://github.com/moj-analytical-services/splink/blob/master/splink/files/settings_jsonschema.json).  




In [8]:
settings = {
    "link_type": "link_only", 
    "max_iterations": 5,
    "blocking_rules": [
        'l.first_name = r.first_name',
        'l.surname = r.surname',
        'l.dob = r.dob'
    ],
    "comparison_columns": [
       {
        "custom_name": "name_inversion",
        "custom_columns_used": ["first_name", "surname", "dob"],
        "case_expression": "CASE WHEN first_name_l = first_name_r AND surname_l = surname_r THEN 2 WHEN first_name_l = surname_r AND surname_l = first_name_r THEN 1 ELSE 0 END",
        "num_levels": 3
        },
        {
            "col_name": "dob"
        },
        {
            "col_name": "city"
        },
        {
            "col_name": "email"
        }
    ],
    "additional_columns_to_retain": ["group"]
    
}

In words, this setting dictionary says:

- We are performing a data linking task (the other options are `dedupe_only`, or `link_and_dedupe`)
- Rather than generate all possible comparisons (the cartesian product of the input datasets), we are going restrict record comparisons to those generated by at least one of the rules in the specified array
- When comparing records, we will use information from the `first_name`, `surname`, `dob`, `city` and `email` columns to compute a match score.
- For `first_name` and `surname`, string comparisons will have three levels:
    - Level 2: Strings are (almost) exactly the same
    - Level 1: Strings are similar 
    - Level 0: No match
- We will make adjustments for term frequencies on the `first_name` and `surname` columns
- We will retain the `group` column in the results even though this is not used as part of comparisons.  This is a labelled dataset and `group` contains the true match - i.e. where group matches, the records pertain to the same person

## Step 4: Save the Two Datasets as Spark Tables

In [9]:
df_l.write.format('parquet').mode("overwrite").saveAsTable('ER_table_left')
df_r.write.format('parquet').mode("overwrite").saveAsTable('ER_table_right')

In [10]:
#spark.catalog.listTables("default")

## Step 4:  Estimate match scores using the Expectation Maximisation algorithm

In [11]:
from splink import Splink

linker = Splink(settings, spark, df_l=df_l, df_r=df_r)
df_e = linker.get_scored_comparisons()

# Later, we will make term frequency adjustments.  
# Persist caches these results in memory, preventing them having to be recomputed when we make these adjustments.
df_e.persist()  

INFO:splink.iterate:Iteration 0 complete
INFO:splink.params:The maximum change in parameters was 0.4944548845291138 for key π_gamma_name_inversion_prob_dist_match_level_0_probability
INFO:splink.iterate:Iteration 1 complete
INFO:splink.params:The maximum change in parameters was 0.054529160261154175 for key π_gamma_city_prob_dist_match_level_0_probability
INFO:splink.iterate:Iteration 2 complete
INFO:splink.params:The maximum change in parameters was 0.026873305439949036 for key π_gamma_city_prob_dist_match_level_0_probability
INFO:splink.iterate:Iteration 3 complete
INFO:splink.params:The maximum change in parameters was 0.016269318759441376 for key π_gamma_email_prob_dist_match_level_0_probability
INFO:splink.iterate:Iteration 4 complete
INFO:splink.params:The maximum change in parameters was 0.013959884643554688 for key π_gamma_email_prob_dist_match_level_1_probability


DataFrame[match_probability: double, unique_id_l: bigint, unique_id_r: bigint, first_name_l: string, first_name_r: string, surname_l: string, surname_r: string, dob_l: string, dob_r: string, gamma_name_inversion: int, prob_gamma_name_inversion_non_match: double, prob_gamma_name_inversion_match: double, gamma_dob: int, prob_gamma_dob_non_match: double, prob_gamma_dob_match: double, city_l: string, city_r: string, gamma_city: int, prob_gamma_city_non_match: double, prob_gamma_city_match: double, email_l: string, email_r: string, gamma_email: int, prob_gamma_email_non_match: double, prob_gamma_email_match: double, group_l: bigint, group_r: bigint, match_key: string]

In [12]:
df_e.write.format('parquet').mode("overwrite").saveAsTable('ER_target')

## Step 5: Inspect results 



In [13]:
# Inspect main dataframe that contains the match scores
df_e.toPandas().sample(5)

,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,surname_l,surname_r,dob_l,dob_r,gamma_name_inversion,prob_gamma_name_inversion_non_match,prob_gamma_name_inversion_match,gamma_dob,prob_gamma_dob_non_match,prob_gamma_dob_match,city_l,city_r,gamma_city,prob_gamma_city_non_match,prob_gamma_city_match,email_l,email_r,gamma_email,prob_gamma_email_non_match,prob_gamma_email_match,group_l,group_r,match_key
54,0.016302,632,100,Noah,Noah,Gibson,None,1987-05-18,1983-07-20,0,0.992115,0.655863,0,0.951687,0.143546,London,Birmingha,0,0.821861,0.218104,avazquez@banks.com,None,-1,1.000000,1.000000,107,20,0
1070,0.002238,352,578,Wallace,Williams,George,George,2018-10-31,1981-08-06,0,0.992115,0.655863,0,0.951687,0.143546,StkeTon--rent,London,0,0.821861,0.218104,jonathan74@glover.com,derek58@gibbs.biz,0,0.985161,0.133354,61,96,1
213,0.016302,960,335,Gabriel,Gabriel,Bartlett,Potter,1973-12-09,1977-12-21,0,0.992115,0.655863,0,0.951687,0.143546,Wolverhampton,Leed,0,0.821861,0.218104,ogomez@robinson-mckinney.com,None,-1,1.000000,1.000000,173,57,0
868,0.002238,721,364,Sienna,Nancy,Taylor,Taylor,2005-06-04,1989-07-25,0,0.992115,0.655863,0,0.951687,0.143546,Liverpool,London,0,0.821861,0.218104,javierfrederick@kelly.org,wagnershane@landry.com,0,0.985161,0.133354,124,62,1
913,0.991286,419,423,Emily,None,Brown,Brown,2005-07-15,2005-07-15,0,0.992115,0.655863,1,0.048313,0.856454,Lndon,London,0,0.821861,0.218104,sarahbrown@mckinney.com,sarahbron@mckinney.com,1,0.014839,0.866646,71,71,1


The `params` property of the `linker` is an object that contains a lot of diagnostic information about how the match probability was computed.  The following cells demonstrate some of its functionality

An alternative representation of the parameters displays them in terms of the effect different values in the comparison vectors have on the match probability:

In [45]:
params.bayes_factor_chart()

alt.Chart(...)

In [46]:
# If charts aren't displaying correctly in your notebook, you can write them to a file (by default splink_charts.html)
params.all_charts_write_html_file("splink_charts.html", overwrite=True)

You can also generate a report which explains how the match probability was computed for an individual comparison row.  

Note that you need to convert the row to a dictionary for this to work

In [17]:
from splink.intuition import intuition_report
row_dict = df_e.toPandas().sample(1).to_dict(orient="records")[0]
print(intuition_report(row_dict, params))


Initial probability of match (prior) = λ = 0.3851017951965332

Comparison of name_inversion.  Values are:
name_inversion_l: Patel, Mohammed , 1989-03-19
name_inversion_r: Mohammed , Patel, 1989-03-19
Comparison has 3 levels
𝛾 for this comparison = gamma_name_inversion = 1
Amongst matches, m = P(𝛾|match) = 0.12372622638940811
Amongst non matches, u = P(𝛾|non-match) = 2.1532594018935924e-06
Bayes factor = m/u = 57459.97267240647
New probability of match (updated belief): 0.9999722124537389

Comparison of dob.  Values are:
dob_l: 1989-03-19
dob_r: 1989-03-19
Comparison has 2 levels
𝛾 for this comparison = gamma_dob = 1
Amongst matches, m = P(𝛾|match) = 0.856454074382782
Amongst non matches, u = P(𝛾|non-match) = 0.048313207924366
Bayes factor = m/u = 17.727120826328797
New probability of match (updated belief): 0.9999984324428572

Comparison of city.  Values are:
city_l: Sheffield
city_r: Sheffield
Comparison has 2 levels
𝛾 for this comparison = gamma_city = 1
Amongst matches, m = P(𝛾|mat

In [41]:
from splink.diagnostics import splink_score_histogram
from pyspark.sql.functions import expr 
splink_score_histogram(df_e.filter(expr('match_probability > 0.001')), spark)

alt.Chart(...)

## Step 6: Create a Custom Atlas Type (Process) reflecting the EM algorithm

First we need to instantiate the connection to Atlas in CDP

In [18]:
import atlasclient

Endpoint, Username and Passoword are stored as CML project variables and passed dynamically

In [19]:
from atlasclient.client import Atlas
client = Atlas(os.environ["atlas_endpoint"], port='', username=os.environ["atlas_username"], password=os.environ["atlas_password"])

Verify the Client connection is working by querying a random Atlas entity

In [20]:
guid = "c845eb62-d85d-4591-8abe-0c31449cdd95"

In [21]:
entity = client.entity_guid(guid)

In [22]:
entity.entity['attributes']

ERROR:atlasclient.base:Missing attr EntityGuid: searchParameters


{'createTime': 1607119259591,
 'description': None,
 'displayName': None,
 'metadata': {'default_project_engine_type': 'legacy_engine',
  'project_description': 'Build an XGBoost model to predict churn using customer telco data.',
  'project_visibility': 'private',
  'service_name': 'mlgov/mlgovapiserver.mlx.cloudera.site@DEMO-AWS.YLCU-ATMI.CLOUDERA.SITE',
  'session_username': 'alexbleakley'},
 'modifiedTime': 1607119259591,
 'name': 'Churn Modeling with XGBoost - alexbleakley',
 'owner': 'alexbleakley',
 'qualifiedName': 'crn:cdp:ml:us-west-1:8a1e15cd-04c2-48aa-8f35-b4a8c11997d3:workspace:ab34ab61-368e-46a2-923e-6c5830585734/affaa215-c794-436c-af69-dcccfffb3d02',
 'replicatedFrom': None,
 'replicatedTo': None,
 'userDescription': None}

Looks like we have successfully established the connection. Next we can create a custom Atlas type (process) reflecting the EM algorithm

In [23]:
typedef_dict = {
    "enumTypes": [],
    "structTypes": [],
    "classificationDefs":[],
    "entityDefs": [{
        "superTypes": ["Process"],
        "name": "EM_algorithm_linkage",
        "description":"custom_type_for_Entity_Resolution",
        "attributeDefs": [{
            "name": "startTime",
            "isOptional": True,
            "isUnique": False,
            "isIndexable": False,
            "typeName":"string",
            "valuesMaxCount":1,
            "cardinality":"SINGLE",
            "valuesMinCount":0
        }]
    }]
}

And we can now register the new type with Atlas. For more on the Atlas type model, please visit this page: https://docs.cloudera.com/runtime/7.2.7/cdp-governance-overview/topics/atlas-metadata-model-overview.html

In [24]:
#Has already run once so will not run again
#client.typedefs.create(data=typedef_dict)

## Step 7: Instantiate the EM algorithm in Atlas along with lineage reflecting our Linkage Job above

Notice: we need to pass the Atlas guid for the two datasets we compared above as they were registered in Atlas when they were stored as a Spark table

In [25]:
#Retrieving GUID's for the three tables via Atlas Client - search by name

In [26]:
#params = {'typeName': 'hive_table', 'attrName': 'data', 'attrValue': 'provider', 'offset': '1', 'limit':'10'}
#search_results = client.search_basic(**params)
#for s in search_results:
#    for e in s.entities:
#        print(e.guid)
#        print(e.attributes)
#        print(e.attributes.values)
#        print(e.typeName)
#        print(e.attributes)

In [27]:
#params = {'typeName': 'hive_table', 'attrName': 'name', 'attrValue': 'cc_data', 'offset': '1', 'limit':'10'}
#search_results = client.search_attribute(**params)
#for s in search_results:
#    for e in s.entities:
#        print(e.guid)
#        print(e.attributes)

In [28]:
#for s in search_results:
#    print(s.entities.to_dict())

In [29]:
#data = {'typeName': 'hive_table', 'attrName': 'name', 'attrValue': 'cc_data', 'offset': '1', 'limit': '100'}
#search_results = client.search_basic.create(data=data)
#for e in search_results.entities:
#    print(e.guid)
#    print(e.attributes)

In [30]:
process_entity_dict = {
  "entity" : {
    "guid" : "-2089428075574333",
    "status" : "ACTIVE",
    "createdBy" : "pdefusco",
    "updatedBy" : "pdefusco",
    "createTime" : "12342",
    "updateTime" : "12342",
    "version" : "12342",
    "relationshipAttributes" : {},
    "classifications" : [],
    "typeName" : "EM_algorithm_linkage",
    "attributes" : {
      "startTime" : "123",
      "qualifiedName": "EM Record Linkage",
      "name":"EM Record Linkage",
      "description":"Record Linkage Algorithm",
      "owner": "pdefusco",
        #, 
      "inputs":[{"guid": "aa955089-5a11-46d9-9dbf-2f6b75f4d65b", "typeName":"hive_table"},
               {"guid": "43d788ce-4af4-4253-af0b-465ea45c1b93", "typeName":"hive_table"}], 
      "outputs":[{"guid":"ac1bdcb3-73c8-4198-a8e6-0aa104c606bb", "type_name":"hive_table"}]
    }, 
  },
  
}

In [31]:
client.entity_post.create(data=process_entity_dict)

{'guidAssignments': {'-2089428075574333': '44848fe5-6950-4a73-a89c-9775b736b4c9'}}

## Step 8: Navigate to Atlas (SDX) and browse for the "EM_algorithm_linkage" entity. Expand the lineage tab and the source and target datasets will be shown

![title](images/ER_atlas_lineage.png)

Next we can optionally remove the EM Algorithm instance from Atlas via the client

In [32]:
entity = client.entity_guid("44848fe5-6950-4a73-a89c-9775b736b4c9")

In [35]:
entity.entity['attributes']["owner"]

'pdefusco'

In [36]:
entity.delete()

## We have completed our introduction to Splink and the Atlas Client. 
## Next we will simulate a real world Application with CML Jobs and COD (Cloudera Operational Database)